In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import random
import statsmodels.api as sm
import operator
from yahoo_finance import Share
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
import pickle

import FinancialData
import TimeSeriesFunctions
import FactorCollection
import BenchmarkReturn

import time                                                
 
def timeit(method): 
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print ('{} {} sec'.format(method.__name__, te-ts))
        return result
    return timed

num_factors = 28

errortickers = []
#QUANDL_TOKEN = ""
#BENCHMARK_INDEX = "SPY"
filename='factor_model'
# CONSTANTS
NUMBER_OF_STOCKS = 100 
TOP_STOCKS = 10 

CLOSE_STR = 'Close'
VOLUME_STR = 'Volume'
OPEN_STR = 'Open'
DATE_STR = 'Date'
LOW_STR = 'Low'
HIGH_STR = 'High'
ADJ_CLOSE_STR = 'Adj_Close'
SYMBOL_STR = 'Symbol'
STOCKS_FILE =  'sp500_names.csv'

insample_start_year=2012 
insample_end_year=2015
outsample_start_year=2015
outsample_end_year=2016

class Factors:
    """Z-scores and Alpha generating factors"""
    # Get returns for a group of tickers
    @timeit
    def __init__(self):
        self.column_name = CLOSE_STR
        tickers = FinancialData.FinancialData.read_stock_names(STOCKS_FILE)
        self.ticker_sample = tickers[:NUMBER_OF_STOCKS]
        print("done getting tickers")
        self.dates_insample = TimeSeriesFunctions.TimeSeriesFunctions.dates_monthly(insample_start_year,insample_end_year)
        self.dates_outsample = TimeSeriesFunctions.TimeSeriesFunctions.dates_monthly(outsample_start_year,outsample_end_year)
        print("done getting dates")
        
        self.dfs = {}
        d1 = pickle.load(open("stock_data.pkl","rb"))
        print("done getting ts data")
        for ticker in self.ticker_sample:
            self.dfs[ticker] = d1[d1['ticker']==ticker]
        print("done populating dfs")
        
        self.returns, failed_tickers = TimeSeriesFunctions.TimeSeriesFunctions.returnsMonthly(self.dfs,
                                                                              self.column_name,
                                                                              self.dates_insample,
                                                                              self.dates_outsample)     
        for t in failed_tickers:
            print("deleting key for {}".format(t))
            del self.dfs[t]
            self.ticker_sample.remove(t)     
        print("avail tickers:")
        for key in list(self.dfs.keys()): 
            print (key)        
        print ("Monthly returns calculated")

    @timeit
    def z_scoring(self,date): # 1. returns list of factors. Each factor has a list of factor values for each ticker
        tickers = list(self.dfs.keys())
        print("z_scoring: getting_factors")
        all_factors = FactorCollection.FactorCollection().allFactors(list(self.dfs.values()),date,self.column_name)[0]
        z_factors = []
        zf_dict = {}
        print("z_scoring: calc z scores")
        for factor in all_factors:  # Calculate factor z-scores => normalize for comparison...
            z_factor = (factor - np.mean(factor)) / np.std(factor)
            z_factors.append(z_factor)
        z_factors = np.array(z_factors)
        where_are_nans = np.isnan(z_factors)
        z_factors[where_are_nans] = 0.
        for i in range(len(tickers)):
            zf_dict[tickers[i]] = z_factors.T[i]
        return zf_dict

    @timeit
    def data_for_regression(self,year): #"""Data for regression for a given year"""
        rs = [];zs = [];ds_dict = {};z_dict = {}
        for key in self.ticker_sample:
            print ("data_for_regression for {}".format(key))
            r = self.returns[key].loc[dt.date(year,1,1):dt.date(year,12,31)]
            rs.append(r['Return'].values)
            ds = r.index.values
            ds_dict[key] = ds
        ds_dates = ds_dict.values()
        ds_dates = [val for sublist in ds_dates for val in sublist]   #????
        ds_dates = list(set(ds_dates))
        for d in ds_dates:
            z_dict[d] = self.z_scoring(d)
        for key in self.ticker_sample:
            for d in ds_dict[key]:
                zs.append(z_dict[d][key])
        rs = np.concatenate(rs)
        zs = np.array(zs)
        return rs, zs  

    @timeit
    def data_for_regression_all(self):
        """Data for regression for all years"""
        dates = TimeSeriesFunctions.TimeSeriesFunctions.dates_monthly(insample_start_year,insample_end_year)
        year1 = pd.to_datetime(dates[0]).year
        year2 = pd.to_datetime(dates[-1]).year
        dr = {}
        for y in range(year1,year2+1):
            dr[y] = self.data_for_regression(y)
            print ('Regression data loaded for year', y)
        return dr

    @timeit
    def regression(self,data_reg):
        years = list(data_reg.keys())
        print("regression", years)
        tvals = {}
        X_all_years = []
        for year in years:
            X = data_reg[year][1]  # Factors ticker1-month [factor1, factor-n], [factor1, factor-n] ticker2-month [],[],[]
            y = data_reg[year][0] # returns ticker [month 1-12] ticker 2 [month 1-12]
            est = sm.OLS(y,X).fit() # how well factor z-scores explain monthly returns via linear regression
            tvals[year] = est.tvalues
            where_are_nans = np.isnan(tvals[year])
            tvals[year][where_are_nans] = 0.
            if year == years[0]:
                X_all_years = X
            else:
                X_all_years = np.concatenate((X_all_years,X))
        #print(np.array(list(tvals.values())).T)
        # mean t-value for each regression X-factor for all years
        mean_t = np.array([np.array(list(tvals.values())).T[i].mean() for i in range(num_factors)])
        # std t-value for each regression X-factor for all years
        std_t = np.array([np.array(list(tvals.values())).T[i].std() for i in range(num_factors)])
        t = mean_t / std_t  # t value sharpe ratio
        #self.t = t
                
        corr = np.corrcoef(X_all_years.T)
        where_are_nans = np.isnan(corr)
        corr[where_are_nans] = 0.
        correlated = []
        for i in range(num_factors):
            for j in range(i+1,num_factors):
                if abs(corr[i,j]) > 0.9:
                    correlated.append(i)
        correlated = set(correlated)       
        print("done regression")
        return t, correlated

    @timeit
    def number_of_factors(self,data_reg):
        print("begin number_of_factors")
        """We leave several factors"""
        UnfilteredFactorZScores, correlated = self.regression(data_reg)   # t = t = mean_t / std_t  # t value "zscore"
        # risk adjusted t value fit for each factor (in regard to all tickers). The higher thebetter.
        # correlated = Factor value correlations over time, only those 90%+ correlated
        # UnfilteredFactorZScores has the same index positions as 
        # UnfilteredFactorZScores = [x,y,z] np array of "sharpe" t/z vals (same as # of factors) with indices 0,1,2,3,4 ... 28
        # correlated = position # of num_factors that are 90%+ correlated??
        
        #UnfilteredFactorZScores.argsort() sort list value indexes in the ASC order
        # np.array([3,1,2,5]).argsort()  ==> RESULT: array([1, 2, 0, 3])
        n = 5
        while True:
            n+=1
            FactorZScoreIndexPosTopFiltered = UnfilteredFactorZScores.argsort()[-n:][::-1]  # Take n-largest values and reverse  [::-1]  reverse sort last becomes 
            # Exclude correlated columns # since we are resorting index positions, then ensure no correlated ones are in
            FactorZScoreIndexPosTopFiltered = list(set(FactorZScoreIndexPosTopFiltered) - correlated)
            if len(FactorZScoreIndexPosTopFiltered)>=6:
                break
        
        # now we have top 6 most significant factors by t-value! (ii - their index positions in t)
        print("done number_of_factors")
        return  FactorZScoreIndexPosTopFiltered, UnfilteredFactorZScores, correlated

    #FactorsClass.monte_carlo(UnfilteredFactorZScores, FactorZScoreIndexPos)  t, ii
    @timeit
    def monte_carlo(self,UnfilteredFactorZScores,FactorZScoreIndexPosTopFiltered): # list of top factor z-scores, their index positions
        dates = TimeSeriesFunctions.TimeSeriesFunctions.dates_monthly(insample_start_year,insample_end_year)
        year1 = pd.to_datetime(dates[0]).year - 1
        year2 = pd.to_datetime(dates[-1]).year + 1
        years = []
        for y in range(year1,year2):
            year = str(dt.date(y,12,31))   # '2016-12-31'
            years.append(year)
        topFactorList_list = []  #f_list
        stockReturns_list = []  # r_list
        stockReturns_dict = {} # r
        topFactorsbyTicker = {} # f = returns and factors in dict
        factors = {}
        print("MC: begin z_scoring") 
        #for i in range(1,len(years)): # as of end of year date xxxx/12/31
        #    factors[years[i]] = self.z_scoring(years[i]) # get dict of factors : [factor vals for each tickers]
            
        #output = open('z_scoring.pkl', 'wb')
        #pickle.dump(factors, output)
        #output.close()
        factors = pickle.load(open("z_scoring.pkl","rb"))    
            
            
      
        print("MC: start tickers")    
        for ticker in self.ticker_sample:
            for i in range(1,len(years)):
                date1 = self.returns[ticker].index.asof(pd.to_datetime(years[i-1])) #most recent date up to "asof"
                date2 = self.returns[ticker].index.asof(pd.to_datetime(years[i]))  
                topFactorList_list.append(factors[years[i]][ticker][FactorZScoreIndexPosTopFiltered])  # select top factors by their index positions 
                stockReturns_list.append((self.returns[ticker].loc[date2]['Return'] + 1) / 
                              (self.returns[ticker].loc[date1]['Return'] + 1) - 1)

            # Normalized z-scores based on mean and std of all years, for each ticker
            topFactorsbyTicker[ticker] = np.array([np.mean(x) for x in np.array(topFactorList_list).T])  / np.array([np.std(x) for x in np.array(topFactorList_list).T])
            where_are_nans = np.isnan(topFactorsbyTicker[ticker])
            topFactorsbyTicker[ticker][where_are_nans] = 0.
            stockReturns_dict[ticker] = np.mean(stockReturns_list) / np.std(stockReturns_list)  # Returns normalized z-score over x-years
            
        # sort by returns ASC   
        print("MC:Finished with ticker normalization")
        sortedStockReturnsByZScore = sorted(stockReturns_dict.items(), key=operator.itemgetter(1))   # function to sort dictionary operator.itemgetter(1)
        # get top average returns (in the form of Z-Scores by all sample stocks; filter top X returns/Z-scores)
        r_top = dict(sortedStockReturnsByZScore[-TOP_STOCKS:])
        topReturnsTickers = list(r_top.keys())  # top_tickers
        # average factor values across all top stocks
        stock_averaged_top_factors = np.array([np.mean(x) for x 
                                               in np.array([topFactorsbyTicker[tt] for tt in topReturnsTickers]).T])

        # Monte-Carlo part  # loadings/weight computations
        print("Started Monte-Carlo loading optimization")
        max_factor = 0.
        for k in range(100000):  
            factorWeights = np.array([random.randint(-2,2) * 0.25 for n in range(len(FactorZScoreIndexPosTopFiltered))])
            mf = sum(stock_averaged_top_factors * factorWeights)
            if mf > max_factor:
                max_factor = mf
        return topReturnsTickers, factorWeights
        # returns optimal allocation weights for top performing factors

    # FactorZScoreIndexPos = = index position of factor z-scores with the highest t-value ("normalized z-scores")
    @timeit
    def generateBacktest(self,factorWeights,FactorZScoreIndexPos):
        print("FactorZScoreIndexPos",FactorZScoreIndexPos)
        print("factorWeights",factorWeights)
        # select top 10 long and top 10 short by in-sample returns
        # test out of sampple
        # print charts
        print("started output function")        
        dates = self.dates_insample + self.dates_outsample
        year1 = pd.to_datetime(dates[0]).year
        year2 = pd.to_datetime(dates[-1]).year
        years = []
        for y in range(year1,year2):
            year = str(dt.date(y,12,31))
            years.append(year)

        print("bench",years)        
        spx_mon_returns,spx_ann_returns = BenchmarkReturn.BenchmarkReturn().getBenchmarkReturn(years,self.column_name,self.dates_insample,self.dates_outsample)

        
        top_returns = []
        long_portfolio = {}
        for date in years:
            print("year:",date)
            allFactors = FactorCollection.FactorCollection().allFactors(
                list(self.dfs.values()),
                date,
                self.column_name)
            allFactorsValues = allFactors[0]
            allFactorsNames = allFactors[1]
            usedFactors = np.array(allFactorsNames)[FactorZScoreIndexPos]
            filteredTopFactors = np.array(allFactorsValues)[FactorZScoreIndexPos].T   # get only top factors
            #print("260 filteredTopFactors[-3:]", filteredTopFactors.shape, filteredTopFactors[-3:])
            where_are_nans = np.isnan(filteredTopFactors)
            filteredTopFactors[where_are_nans] = 0.
            print("filteredTopFactors",filteredTopFactors)
            # Final unified factor # LONG PORTFOLIO COMPOSITION
            PortfolioFactorWeights = np.array([x.sum() for x in (filteredTopFactors * factorWeights)])  #???
            # summed all positive and negative factors*weights for every stock in the universe
            #print("PortfolioFactorWeights",PortfolioFactorWeights)
            print("len(PortfolioFactorWeights)",len(PortfolioFactorWeights), PortfolioFactorWeights.shape)
            top_stocks_idx = PortfolioFactorWeights.argsort()[-TOP_STOCKS:][::-1] 
            # stocks with the biggest positive factor loadings / weights have the highest correlation to the 
            # top factors that have the highest correlation to returns ( def number_of_factors(self,data_reg):)
            print("top_stocks_idx",top_stocks_idx)
            top_stocks = np.array(self.ticker_sample)[top_stocks_idx].tolist()
            long_portfolio[date] = {"tickers":top_stocks, "weights":[1/TOP_STOCKS]*TOP_STOCKS,
                                    "PortfolioFactors":usedFactors,
                                   "PortfolioFactorWeights":PortfolioFactorWeights[top_stocks_idx]}
            print("top_stocks",top_stocks)
            top_dfs = {k:self.dfs[k] for k in top_stocks if k in self.dfs} # sub-dictionary of dfs
            # get monthly returns of those stocks - "proxies" to those factors
            top_returns_monthly = TimeSeriesFunctions.TimeSeriesFunctions.returnsMonthly(
                top_dfs,
                self.column_name,
                self.dates_insample, 
                self.dates_outsample)
            print("top_returns_monthly")
            
            tr = 0.
            for s in top_stocks:
                # SUM monthly returns for each stock
                #latest_ret = top_returns_monthly[0].get(s).loc[top_returns_monthly[0].
                #                                       get(s).index.asof(pd.to_datetime(date))]['Return']
                monthly_ret = top_returns_monthly[0].get(s)
                monthly_ret = monthly_ret.groupby(monthly_ret.index).first().dropna()
                annual_ret = monthly_ret.groupby(pd.TimeGrouper("A")).sum().dropna()
                
                #print(s,type(annual_ret),annual_ret)
                tr += annual_ret.iloc[0]["Return"]
                #print("growing tr:",tr)
            tr = tr / TOP_STOCKS 
            print("past averaging return",tr,TOP_STOCKS )
            top_returns.append(tr)
            print ('Output generated as of', date[:4])
        # the above is wrong: we need annnual return not December YYYY return!
        returns_stocks = pd.DataFrame(top_returns,index=years,columns=['Return'])
        print("returns_stocks",returns_stocks[-10:])
        print("SPX",spx_ann_returns[-10:])
        print("long_portfolio",long_portfolio)
        #Which factors & their weights:
        print(factorWeights,FactorZScoreIndexPos)
        
        #Cumulative return growth in USD portfolio vs SPX
        #df2.ix[:,0] = np.cumprod(r0 + 1) * 100
        #df2.ix[:,1] = np.cumprod(r1 + 1) * 100

if __name__ == "__main__":
    print ('Simulations STARTED...')
    #FactorsClass = Factors() # class initialisation
    #output = open('FactorsClass1.pkl', 'wb')
    #pickle.dump(FactorsClass, output)
    #output.close()
    FactorsClass = pickle.load(open("FactorsClass1.pkl","rb"))
    
    print ("Assembling data for regression")
    #data_reg = FactorsClass.data_for_regression_all() # assembling data for regression
    #output = open('data_reg1.pkl', 'wb')
    #pickle.dump(data_reg, output)
    #output.close()
    data_reg = pickle.load(open("data_reg1.pkl","rb"))
    
    print ("Reducing number of factors")
    # FactorZScoreIndexPos = index position of factor z-scores with the highest t-value ("normalized z-scores") (and factors too)
    # UnfilteredFactorZScores = #list of t-value z-scores for each factor (in the factor order as well)
    # correlated = 
    FactorZScoreIndexPos, UnfilteredFactorZScores,correlated = FactorsClass.number_of_factors(data_reg) # we leave several factors and doing regression

    print ("Doing simulations") # Let's load most significant factors
    # top_tickers = 
    # weights = 
    top_tickers, weights = FactorsClass.monte_carlo(UnfilteredFactorZScores, FactorZScoreIndexPos) # monte-carlo simulations
    print("top_tickers, weights", top_tickers, weights)
    FactorsClass.generateBacktest(weights,FactorZScoreIndexPos) # output
    print ('Simulations DONE.')
    
    # Generate monthly portfolio allocations with ticker / weights (First Long, then L/S)

Simulations STARTED...
Assembling data for regression
Reducing number of factors
begin number_of_factors
regression [2012, 2013, 2014]
done regression
regression 0.032958269119262695 sec
done number_of_factors
number_of_factors 0.03382110595703125 sec
Doing simulations
MC: begin z_scoring
MC: start tickers


/usr/local/lib/python3.4/dist-packages/numpy/lib/function_base.py:3003: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.4/dist-packages/numpy/lib/function_base.py:3004: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


MC:Finished with ticker normalization
Started Monte-Carlo loading optimization
monte_carlo 2.5015981197357178 sec
top_tickers, weights ['HST', 'ADM', 'MUR', 'APD', 'TGT', 'DUK', 'VLO', 'CLX', 'NSC', 'ABT'] [-0.5  0.   0.   0.5  0.   0.5]
FactorZScoreIndexPos [0, 1, 2, 10, 13, 17]
factorWeights [-0.5  0.   0.   0.5  0.   0.5]
started output function
bench ['2012-12-31', '2013-12-31', '2014-12-31']
year: 2012-12-31


/home/ubuntu/portfolio/factors/FactorCollection.py:34: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=0,span=50,adjust=True,ignore_na=False).mean()
  ema = pd.ewma(cut_df[column_name], span=50).ix[-25:]
/home/ubuntu/portfolio/factors/FactorCollection.py:117: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=0,span=10,adjust=True,ignore_na=False).mean()
  ema = pd.ewma(cut_df[column_name], span=10).ix[-5:]
/home/ubuntu/portfolio/factors/FactorCollection.py:127: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=0,span=300,adjust=True,ignore_na=False).mean()
  ema = pd.ewma(cut_df[column_name], span=300).ix[-150:]
/home/ubuntu/portfolio/factors/FactorCollection.py:103: RuntimeWarning: invalid value encountered in double_scalars
  return (((close -

filteredTopFactors [[  3.01466164e-03  -1.22928291e+00  -1.88004723e+00   2.88295062e+06
    5.44444444e-01   1.00070340e+00]
 [ -2.71776159e-03  -7.06306366e-01   2.02502127e+00   9.62360000e+06
    4.44444444e-01   9.29902396e-01]
 [  1.42403237e-02  -1.71540239e+00  -1.59725204e+00   8.71331148e+05
    4.83333333e-01   9.91251238e-01]
 [  2.35011854e-02   1.26043744e+00   3.12563223e-01   2.00454187e+07
    5.00000000e-01   1.13100437e+00]
 [ -5.42776933e-02  -9.73187841e-01  -3.10233033e+00   5.95685484e+06
    5.16666667e-01   9.47375729e-01]
 [  5.46362450e-02  -1.79321233e+00   4.13089305e+00   2.06769231e+06
    6.38888889e-01   1.00588502e+00]
 [  3.58399187e-03  -1.04606543e+00   3.72026024e+00   1.60226298e+06
    5.33333333e-01   9.63502304e-01]
 [  1.10037764e-04  -1.26151477e+00   5.58153664e-01   4.32371558e+06
    5.16666667e-01   9.74115412e-01]
 [  4.60035018e-02  -1.11562046e+00   4.01625162e-01   6.33038690e+06
    5.38888889e-01   1.05124941e+00]
 [  9.65725868e-02